## NOTE:
- The treatment variable definitely isn't set up correctly in the linear regression, but this is a good baseline to work off of. If anyone has any ideas on how to improve feel free to try stuff.
- I'm not certain what th efinal output should look like, so I don't really know how close I am

### Prelude:

In [100]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import statsmodels.api as sm
set_seed = 42

In [101]:
data = pd.read_csv("./../Datasets/cleaned_data.csv")

In [102]:
#from Tiffanie's code
# distribution of subjects across treatment conditions (like Table 1 from paper)
# N = 186
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.118280
1,Positive science,22,0.118280
2,Negative science,18,0.096774
3,Religious,35,0.188172
4,Equity,30,0.161290
5,Efficiency,32,0.172043
6,Secular,27,0.145161


### Difference of Means Tables:

In [103]:
pd.pivot_table(data, values=['mean_climate_support_before', 'mean_climate_support_after', 'mean_climate_support_change'],
               index=['treatment_value','treatment_frame'],
               aggfunc=['mean'])

mean  \
                                 mean_climate_support_after   
treatment_value treatment_frame                               
0               No framing                         1.590909   
1               Positive science                   1.795455   
2               Negative science                   1.972222   
3               Religious                          1.857143   
4               Equity                             1.916667   
5               Efficiency                         1.953125   
6               Secular                            1.990741   

                                                              \
                                 mean_climate_support_before   
treatment_value treatment_frame                                
0               No framing                          1.545455   
1               Positive science                    1.670455   
2               Negative science                    1.763889   
3               Religious                           1.750000   
4               Equity                              1.866667   
5               Efficiency                          1.914062   
6               Secular                             1.879630   

                                                              
                                 mean_climate_support_change  
treatment_value treatment_frame                               
0               No framing                          0.045455  
1               Positive science                    0.125000  
2               Negative science                    0.208333  
3               Religious                           0.107143  
4               Equity                              0.050000  
5               Efficiency                          0.039062  
6               Secular                             0.111111

In [104]:
pd.pivot_table(data, values=["mean_climate_support_after", 'mean_climate_support_before', 'mean_climate_support_change'],
               index=["party_id", "treatment_frame"], aggfunc=['mean'])

mean  \
                               mean_climate_support_after   
party_id      treatment_frame                               
Democrat      Efficiency                         2.333333   
              Equity                             1.946429   
              Negative science                   2.550000   
              No framing                         1.725000   
              Positive science                   2.343750   
              Religious                          2.026316   
              Secular                            1.980769   
Independent   Efficiency                         1.946429   
              Equity                             1.900000   
              Negative science                   2.062500   
              No framing                         1.725000   
              Positive science                   1.468750   
              Religious                          1.694444   
              Secular                            2.000000   
No preference Efficiency                         1.750000   
              Equity                             2.333333   
              No framing                         0.500000   
              Positive science                   0.666667   
              Religious                          1.916667   
              Secular                            2.500000   
Republican    Efficiency                         1.100000   
              Equity                             1.718750   
              Negative science                   0.437500   
              No framing                         0.000000   
              Positive science                   2.333333   
              Religious                          1.375000   
              Secular                            1.666667   

                                                            \
                               mean_climate_support_before   
party_id      treatment_frame                                
Democrat      Efficiency                          2.270833   
              Equity                              1.946429   
              Negative science                    2.200000   
              No framing                          1.600000   
              Positive science                    2.187500   
              Religious                           1.947368   
              Secular                             1.865385   
Independent   Efficiency                          1.803571   
              Equity                              1.900000   
              Negative science                    2.000000   
              No framing                          1.700000   
              Positive science                    1.375000   
              Religious                           1.666667   
              Secular                             1.916667   
No preference Efficiency                          1.750000   
              Equity                              2.250000   
              No framing                          1.000000   
              Positive science                    0.666667   
              Religious                           1.750000   
              Secular                             2.500000   
Republican    Efficiency                          1.400000   
              Equity                              1.562500   
              Negative science                    0.437500   
              No framing                          0.000000   
              Positive science                    2.083333   
              Religious                           1.000000   
              Secular                             1.416667   

                                                            
                               mean_climate_support_change  
party_id      treatment_frame                               
Democrat      Efficiency                          0.062500  
              Equity                              0.000000  
              Negative science                    0.3500

### Linear Regression:

In [105]:
# Function to perform Lin's Estimator regression with multiple treatment indicators
def lin_estimator_mult_treat(data, y_var, treatment_var, covariate_list):
    '''linear regression with robust standard errors'''
    # Demean the covariates
    for cov in covariate_list:
        data[cov + '_demeaned'] = data[cov] - data[cov].mean()

    # Create interaction terms for each treatment and each demeaned covariate
    for treat in treatment_var:
        for cov in covariate_list:
            data[treat + '_X_' + cov] = data[treat] * data[cov + '_demeaned']

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_var)
    
    # Include each demeaned covariate
    covariates_formula = " + ".join([cov + '_demeaned' for cov in covariate_list])
    
    # Include each interaction term
    interactions_formula = " + ".join([treat + '_X_' + cov for treat in treatment_var for cov in covariate_list])

    # Full formula
    formula = f"{y_var} ~ {treatments_formula} + {covariates_formula} + {interactions_formula}"

    # Fit the regression model
    model = sm.OLS.from_formula(formula, data=data).fit()

    return model.get_robustcov_results(cov_type="HC3")


In [106]:
#NOT SURE IF THESE ARE RIGHT (ESPECIALLY THE TREATMENT VALUES) - NEED TO CHECK
# IF YOU EDIT THESE IT WILL EDIT INPUT FOR ALL REGRESSIONS
outcome_var = 'mean_climate_support_after'
covariates = ['Religiosity', 'Economic_Reasoning', 'ScientificConfidence']
treatment_vars = ['treatment_value']

In [107]:
result = lin_estimator_mult_treat(data, outcome_var, treatment_vars, covariates).summary()
print(result)

                                OLS Regression Results                                
Dep. Variable:     mean_climate_support_after   R-squared:                       0.185
Model:                                    OLS   Adj. R-squared:                  0.153
Method:                         Least Squares   F-statistic:                     4.847
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           5.09e-05
Time:                                00:31:19   Log-Likelihood:                -190.51
No. Observations:                         185   AIC:                             397.0
Df Residuals:                             177   BIC:                             422.8
Df Model:                                   7                                         
Covariance Type:                          HC3                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

### Cross-Validation Regression:

In [108]:
#Randomly split data into two folds
shuffled = data.sample(frac=1, random_state=set_seed)
split = np.array_split(shuffled, 2)
fold1 = split[0]
fold2 = split[1]

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [109]:
def find_best_treatment(effects):
    # Find the treatment with the maximum effect size
    return max(effects, key=effects.get)

def assign_best_treatment_indicator(data, best_treatment):
    data['best_treatment_indicator'] = (data[treatment_vars] == best_treatment).astype(int)

def estimate_best_treatment_effect(data, outcome, indicator, covariates):
    formula = f"{outcome} ~ {indicator} + {' + '.join(covariates)}"
    model = sm.OLS.from_formula(formula, data=data).fit()
    return model.summary()

def extract_treatment_effects(model, treatment_vars):
    coefs = dict(zip(model.model.exog_names, model.params))
    return {var: coefs[var] for var in treatment_vars if var in coefs}
 

In [110]:
# Estimate effects in each fold
effects_fold1 = extract_treatment_effects(lin_estimator_mult_treat(fold1, outcome_var, treatment_vars, covariates), treatment_vars)
effects_fold2 = extract_treatment_effects(lin_estimator_mult_treat(fold2, outcome_var, treatment_vars, covariates), treatment_vars)

# Find best treatments
best_treatment_fold1 = find_best_treatment(effects_fold1)
best_treatment_fold2 = find_best_treatment(effects_fold2)

# Assign indicators based on the best treatment from the opposite fold
assign_best_treatment_indicator(fold1, best_treatment_fold2)
assign_best_treatment_indicator(fold2, best_treatment_fold1)

# Combine folds back
combined_data = pd.concat([fold1, fold2])

# Estimate the effect of being in the best treatment condition
final_result = estimate_best_treatment_effect(combined_data, outcome_var, 'best_treatment_indicator', covariates)
print(final_result)


                                OLS Regression Results                                
Dep. Variable:     mean_climate_support_after   R-squared:                       0.155
Model:                                    OLS   Adj. R-squared:                  0.141
Method:                         Least Squares   F-statistic:                     11.05
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           1.07e-06
Time:                                00:31:19   Log-Likelihood:                -193.91
No. Observations:                         185   AIC:                             395.8
Df Residuals:                             181   BIC:                             408.7
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

### Random Forest:

In [111]:
features = covariates + treatment_vars
folds = [fold1, fold2]

# Train models and predict outcomes
def train_and_predict(train_data, test_data):
    predictions = {}

    # Train a model for each treatment condition and predict for the test data
    for treatment in treatment_vars:
        # Assuming binary treatment, filter data where treatment is active
        treated_data = train_data[train_data[treatment] == 1]
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(treated_data[covariates], treated_data[outcome_var])
        # Store predictions for each treatment
        predictions[treatment] = rf.predict(test_data[covariates])

    return predictions

# Analyze both folds
results_fold1 = train_and_predict(fold2, fold1)  # Train on fold2, predict for fold1
results_fold2 = train_and_predict(fold1, fold2)  # Train on fold1, predict for fold2

def assign_best_condition(test_data, results):
    # Create a DataFrame from the results with appropriate indexing
    results_df = pd.DataFrame(results, index=test_data.index)
    # Use np.argmax on axis=1 to find the indices of maximum values along the horizontal axis
    best_treatment_indices = np.argmax(results_df.values, axis=1)
    # Convert indices to a Series to use the map function
    best_treatment_series = pd.Series(best_treatment_indices, index=test_data.index)
    # Map indices to treatment variable names
    best_treatment = best_treatment_series.map(dict(enumerate(treatment_vars)))
    test_data['best_personalized_treatment'] = best_treatment
    test_data['best_treatment_indicator'] = 1  # This assumes the personalized treatment is applied


# Assign best treatment based on opposite fold predictions
assign_best_condition(fold1, results_fold2)
assign_best_condition(fold2, results_fold1)

# Estimate the effect of the best personalized condition
def estimate_personalized_effect(data, treatment_indicator, outcome, covariates):
    formula = f"{outcome} ~ {treatment_indicator} + {' + '.join(covariates)}"
    model = sm.OLS.from_formula(formula, data=data).fit()
    return model.summary()

# Combining data for final analysis
combined_data = pd.concat([fold1, fold2])
effect_summary = estimate_personalized_effect(combined_data, 'best_treatment_indicator', outcome_var, covariates)
print(effect_summary)

                                OLS Regression Results                                
Dep. Variable:     mean_climate_support_after   R-squared:                       0.155
Model:                                    OLS   Adj. R-squared:                  0.141
Method:                         Least Squares   F-statistic:                     11.05
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           1.07e-06
Time:                                00:31:19   Log-Likelihood:                -193.91
No. Observations:                         185   AIC:                             395.8
Df Residuals:                             181   BIC:                             408.7
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------